## Deploy endpoint

In [9]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice 
from azureml.core.model import Model
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: capstone
Azure region: westeurope
Subscription id: 72f46e0e-1451-4b79-92cd-fc8f7797bda7
Resource group: test-Jesse


In [76]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

# env = Environment.from_conda_specification(name = 'project1', file_path = '.azureml/project1.yml')
env = Environment.get(workspace=ws, name="AzureML-AutoML")
for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled = True, enable_app_insights = True)

In [77]:
model = Model(ws, name='automl')
service = Model.deploy(workspace=ws,
                        name = 'automl-auth',
                        models= [model],
                        inference_config=inference_config,
                        deployment_config=deployment_config
)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
scoring URI: http://d95e931a-f942-4a3c-b8ca-649d7e5607ed.westeurope.azurecontainer.io/score


## Get testset

In [3]:
import pandas as pd
from sklearn.impute import SimpleImputer
train_url = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/housing_after_preprocessing.csv"
train_x = pd.read_csv(train_url)
train_x.rename(columns={'Unnamed: 0':'Column1'}, inplace=True)
train_x = train_x.drop(columns=['y'])

class DFSimpleImputer(SimpleImputer):
    # just like SimpleImputer, but retuns a df
    # this approach creates problems with the add_indicator=True, since more columns are returned
    # so don't set add_indicator to True
    def transform(self, X,y=None):
        return pd.DataFrame(super().transform(X),columns=X.columns) 
    def __repr__(self):
        return f'SimpleImputer'
imp = DFSimpleImputer(strategy='most_frequent')
imp = imp.fit(train_x)

In [4]:
test_url = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/final_test.csv"
test_x = pd.read_csv(test_url)
test_x.rename(columns={'Unnamed: 0':'Column1'}, inplace=True)
test_x = imp.transform(test_x)
test_x = test_x.astype(train_x.dtypes.to_dict())

In [5]:
# def submit(model, filename='submission.csv'):
#     pred = model.predict(test_x)
#     out = pd.DataFrame()
#     out['Id']= list(range(1461, 2920))
#     out['SalePrice'] = np.exp(pred)
#     out.to_csv('./submissions/'+ filename, index=False)


In [10]:
from azureml.core import Workspace, Experiment, Model
import joblib
model = Model(workspace=ws, name='automl')
model.download(target_dir='models/automl', exist_ok = True)


'models\\automl\\outputs'

In [11]:
m = joblib.load('models/automl/outputs/model.pkl')

In [22]:
data = json.dumps(test_x.to_json())
data = json.loads(data)
data = pd.read_json(data)
preds = m.predict(data)

In [23]:
len(preds), preds[0]

(1459, 11.6830587743674)

## Query endpoint

In [20]:
import pandas as pd
import json
import requests 
service_uri = 'http://d95e931a-f942-4a3c-b8ca-649d7e5607ed.westeurope.azurecontainer.io/score'
key = '2NWoRs2C7IWT0GSdfl1eFy6QAihU7vln'

headers = {'Content-Type':'application/json'}
headers['Authorization'] = f'Bearer {key}'
data = json.dumps(test_x.to_json())
# data = json.loads(data)
response = requests.post(service_uri, data=data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())


1135, 12.646631518399762, 11.902444063788316, 11.007191245895118, 12.645376831658119, 11.177122288044012, 12.437522033526239, 11.860418017617128, 12.103797860638895, 11.935027943018675, 12.892940515761032, 12.655620235368264, 12.283064225848038, 12.35765364609345, 12.174314150039594, 12.861704208424152, 11.792015722492465, 11.965154271660802, 11.732432381701829, 11.779425224244385, 11.866776554842495, 11.827804708250351, 12.127483994989152, 12.079075957735935, 12.0151607386514, 12.147337795160896, 12.098908194153703, 12.04113800914429, 12.382238476691946, 12.148015264157898, 12.005041028570814, 12.130926757491682, 12.24325380934363, 12.763411470710182, 12.827165269180185, 11.884836578617062, 12.627830161912133, 11.987230316950415, 12.471250611272445, 12.162274244121964, 12.461808824777831, 12.34352661505258, 12.048486301650538, 12.130815729393131, 11.808576198418669, 12.691333769441922, 11.932978500891164, 12.631395184714115, 11.881109217960073, 11.644868596719421, 11.71021890631128, 1

In [21]:
len(response.json()['predictions']), response.json()['predictions'][0]

(1459, 11.6830587743674)